In [ ]:
import os
import tensorflow as tf

from omegaconf import OmegaConf

from processing.preprocessing import generate_categories_dict
from processing.postprocessing import create_color_map

from models.model_functions import load_model
from models.saving import load_params
from utils.dir_processing import save_metadata
from utils.utils import create_metadata

from models import evaluate
from models.train_test.test import test_model

from processing.preprocessing import preprocess_data_from_images_dev
from sklearn.model_selection import train_test_split

from models.saving import save_predictions

from processing.postprocessing import encode_masks_to_rgb
from utils.dir_processing import clean_folder

Set up some environment variables

In [ ]:
# Check if laptop has a GPU available for computation
physical_devices = tf.config.experimental.list_physical_devices('GPU')

if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Disabling logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Load configs:

In [ ]:
# environment variables
cfg = OmegaConf.load('configs/env.yaml')

categories = cfg.MODEL.categories
categories_dict = generate_categories_dict(categories)
color_map = create_color_map(categories)

num_classes = len(categories) + 1

desired_input_dimensions = eval(cfg.DATA.img_dims)

Load model or create a new one:

In [ ]:
model_name = cfg.MODEL.model_name

model = load_model(model_name)
load_params(model_name)

PATH_LOG = os.path.join(cfg.DIRS.history, model_name)  # store model log
PATH_RES = os.path.join(cfg.DIRS.results, model_name)  # store results

os.makedirs(PATH_LOG, exist_ok=True)
os.makedirs(os.path.join(PATH_RES, model_name), exist_ok=True)

metadata = create_metadata(model_name)

model.summary()

Split data

In [ ]:
# === DATASET LOADING AND PREPROCESSING === #
X, y = preprocess_data_from_images_dev(data_path = cfg.DIRS.data, 
                                       shape = desired_input_dimensions,
                                       categories=categories,
                                       hspectral = [cfg.HYPERSPECTRAL.hyperspec, cfg.HYPERSPECTRAL.pca])

#=== TRAIN/TEST SPLIT === #
test_size = 1.- cfg.DATA.train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=test_size,
                                                    shuffle=True,
                                                    random_state=cfg.DATA.seed)

print(f'Number of TRAIN images: {len(X_train)}')
print(f'Number of TEST images: {len(X_test)}')

Test model

In [ ]:
# == INFERENCE == #
y_pred = test_model(model, X_test, prediction_threshold=0.8)

# == Confusion matrices == #
confusion_classes, imgs_labels = evaluate.get_confusion_indices(y_test,
                                                                y_pred,
                                                                categories_dict=categories_dict,
                                                                pixel_thres=10,
                                                                meanIoU_threshold=0.7)            
            
for class_name,confusion_matrix in confusion_classes.items():        
    evaluate.save_confusion_matrix(confusion_matrix, model_name, class_name, class_counter=None)

clean_folder(PATH_RES)

# encode ground truth and prediction masks
y_test_en, y_pred_en = encode_masks_to_rgb(y_test, y_pred, color_map)
save_predictions(X_test, 
                 y_test_en,
                 y_pred_en,
                 PATH_RES,
                 imgs_labels,
                 confusion_classes,
                 color_map)